In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression

import os

In [2]:
## 獲取當前工作路徑
current_directory = os.getcwd()

In [3]:
def ma_pre(oven_id, month):
    input_path = os.path.join(current_directory, 'data/accumulation_hour_data', f'group_{oven_id}.csv')
    df = pd.read_csv(input_path)

    # 將日期轉換為日期時間對象
    df['date'] = pd.to_datetime(df['date'])

    grouped = df.groupby('layer_id')

    # 創建一個字典，每個鍵對應一個 layer_id，值是對應的子資料集
    sub_datasets = {}

    # 迭代每個分組，並將分組後的資料存儲在字典中
    for group_name, group_data in grouped:
        sub_datasets[group_name] = group_data
        sub_datasets[group_name].sort_values(by='date', inplace=True)

    new_finished_layer = []
    new_df = pd.DataFrame()
    for layer_id in sub_datasets:
        new_df = pd.DataFrame()
        df_layer = sub_datasets[layer_id]

        df_layer['time_interval'] = df_layer['date'].diff()
        df_layer['accumulation_diff'] = df_layer['accumulation_hour'].diff()
        # Drop the first row
        df_layer = df_layer.iloc[1:]

        # Reset the index if you want
        df_layer.reset_index(drop=True, inplace=True)

        # # 過濾出accumulation_diff大於0的列
        df_layer_processed = df_layer[df_layer['accumulation_diff'] > 0]

        # 將時間間隔列轉換為整數（以天為單位）
        df_layer_processed['time_interval'] = df_layer_processed['time_interval'].dt.days

        # 創建線性回歸模型
        model = LinearRegression()

        # 訓練線性回歸模型
        X = df_layer_processed[['time_interval']]
        y = df_layer_processed['accumulation_diff']
        model.fit(X, y)


        # 循環預測新的time_interval並添加到DataFrame
        while True:
            # 在這裡進行時間間隔預測，假設新的時間間隔為new_time_interval

            # 提取time_interval列的後三個元素並轉換為數值型
            last_three_time_intervals = df_layer_processed['time_interval'].tail(3)

            # 計算平均值
            average_time_interval = last_three_time_intervals.sum() / 3

            new_time_interval = int(average_time_interval.round())

            # 獲取最後一个日期
            last_date = pd.to_datetime(df_layer_processed['date'].iloc[-1])

            # 计算下一个日期
            next_date = last_date + pd.DateOffset(days=new_time_interval)
            if next_date.month >= month:
                break
            # 判斷是否達到了停止條件，这里假設停止條件是日期超過10月
            predicted_accumulation_hour_diff = model.predict([[new_time_interval]])[0]

            predicted_accumulation_hour = df_layer_processed['accumulation_hour'].iloc[-1] + predicted_accumulation_hour_diff

            # 將新數據添加到DataFrame
            df_layer_processed = df_layer_processed.append({'date': next_date.strftime('%Y-%m-%d'),'oven_id': oven_id, 'layer_id': layer_id, 'accumulation_hour': predicted_accumulation_hour, 'time_interval': new_time_interval, 'accumulation_diff': predicted_accumulation_hour_diff}, ignore_index=True)
            new_df = new_df.append({'date': next_date.strftime('%Y-%m-%d'),'oven_id': oven_id, 'layer_id': layer_id, 'accumulation_hour': predicted_accumulation_hour, 'time_interval': new_time_interval, 'accumulation_diff': predicted_accumulation_hour_diff}, ignore_index=True)

        new_finished_layer.append(new_df)
    df_concat = pd.concat(new_finished_layer)
    
    output_path = os.path.join(current_directory, 'data/accmulation_hour_prediction', f'{oven_id}.csv')
    df_concat.to_csv(output_path)
    print(f'{oven_id} finished')

In [4]:
alp = ['B', 'C', 'D', 'E', 'G']
oven_id_lst_p1 = [f'1{x}0' for x in alp]
oven_id_lst_p2 = [f'2{x}0' for x in alp]

In [5]:
for oven_id in oven_id_lst_p1:
    ma_pre(oven_id, 10)

c:\Users\cherr\python\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\cherr\AppData\Local\Temp\ipykernel_16344\27521094.py:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_layer_processed = df_layer_processed.append({'date': next_date.strftime('%Y-%m-%d'),'oven_id': oven_id, 'layer_id': layer_id, 'accumulation_hour': predicted_accumulation_hour, 'time_interval': new_time_interval, 'accumulation_diff': predicted_accumulation_hour_diff}, ignore_index=True)
C:\Users\cherr\AppData\Local\Temp\ipykernel_16344\27521094.py:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append({'date': next_date.strftime('%Y-%m-%d'),'oven_id': oven_id, 'layer_id': layer_id, 'accumulation_hour': pre

1B0 finished


c:\Users\cherr\python\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\cherr\AppData\Local\Temp\ipykernel_16344\27521094.py:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_layer_processed = df_layer_processed.append({'date': next_date.strftime('%Y-%m-%d'),'oven_id': oven_id, 'layer_id': layer_id, 'accumulation_hour': predicted_accumulation_hour, 'time_interval': new_time_interval, 'accumulation_diff': predicted_accumulation_hour_diff}, ignore_index=True)
C:\Users\cherr\AppData\Local\Temp\ipykernel_16344\27521094.py:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append({'date': next_date.strftime('%Y-%m-%d'),'oven_id': oven_id, 'layer_id': layer_id, 'accumulation_hour': pre

1C0 finished


c:\Users\cherr\python\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\cherr\AppData\Local\Temp\ipykernel_16344\27521094.py:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_layer_processed = df_layer_processed.append({'date': next_date.strftime('%Y-%m-%d'),'oven_id': oven_id, 'layer_id': layer_id, 'accumulation_hour': predicted_accumulation_hour, 'time_interval': new_time_interval, 'accumulation_diff': predicted_accumulation_hour_diff}, ignore_index=True)
C:\Users\cherr\AppData\Local\Temp\ipykernel_16344\27521094.py:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append({'date': next_date.strftime('%Y-%m-%d'),'oven_id': oven_id, 'layer_id': layer_id, 'accumulation_hour': pre

1D0 finished


C:\Users\cherr\AppData\Local\Temp\ipykernel_16344\27521094.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_layer_processed['time_interval'] = df_layer_processed['time_interval'].dt.days
c:\Users\cherr\python\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\cherr\AppData\Local\Temp\ipykernel_16344\27521094.py:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_layer_processed = df_layer_processed.append({'date': next_date.strftime('%Y-%m-%d'),'oven_id': oven_id, 'layer_id': layer_id, 'accumulation_hour': predicted_accumulation_hour, 'ti

1E0 finished


C:\Users\cherr\AppData\Local\Temp\ipykernel_16344\27521094.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_layer_processed['time_interval'] = df_layer_processed['time_interval'].dt.days
c:\Users\cherr\python\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\cherr\AppData\Local\Temp\ipykernel_16344\27521094.py:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_layer_processed = df_layer_processed.append({'date': next_date.strftime('%Y-%m-%d'),'oven_id': oven_id, 'layer_id': layer_id, 'accumulation_hour': predicted_accumulation_hour, 'ti

1G0 finished


In [6]:
for oven_id in oven_id_lst_p2:
    ma_pre(oven_id, 3)

C:\Users\cherr\AppData\Local\Temp\ipykernel_16344\27521094.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_layer_processed['time_interval'] = df_layer_processed['time_interval'].dt.days
C:\Users\cherr\AppData\Local\Temp\ipykernel_16344\27521094.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_layer_processed['time_interval'] = df_layer_processed['time_interval'].dt.days
C:\Users\cherr\AppData\Local\Temp\ipykernel_16344\27521094.py:36: SettingWithCopyWarning: 
A value is trying to be set on a 

2B0 finished


C:\Users\cherr\AppData\Local\Temp\ipykernel_16344\27521094.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_layer_processed['time_interval'] = df_layer_processed['time_interval'].dt.days
c:\Users\cherr\python\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\cherr\AppData\Local\Temp\ipykernel_16344\27521094.py:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_layer_processed = df_layer_processed.append({'date': next_date.strftime('%Y-%m-%d'),'oven_id': oven_id, 'layer_id': layer_id, 'accumulation_hour': predicted_accumulation_hour, 'ti

2C0 finished


C:\Users\cherr\AppData\Local\Temp\ipykernel_16344\27521094.py:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append({'date': next_date.strftime('%Y-%m-%d'),'oven_id': oven_id, 'layer_id': layer_id, 'accumulation_hour': predicted_accumulation_hour, 'time_interval': new_time_interval, 'accumulation_diff': predicted_accumulation_hour_diff}, ignore_index=True)
C:\Users\cherr\AppData\Local\Temp\ipykernel_16344\27521094.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_layer_processed['time_interval'] = df_layer_processed['time_interval'].dt.days
c:\Users\cherr\python\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid

2D0 finished


C:\Users\cherr\AppData\Local\Temp\ipykernel_16344\27521094.py:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append({'date': next_date.strftime('%Y-%m-%d'),'oven_id': oven_id, 'layer_id': layer_id, 'accumulation_hour': predicted_accumulation_hour, 'time_interval': new_time_interval, 'accumulation_diff': predicted_accumulation_hour_diff}, ignore_index=True)
c:\Users\cherr\python\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\cherr\AppData\Local\Temp\ipykernel_16344\27521094.py:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_layer_processed = df_layer_processed.append({'date': next_date.strftime('%Y-%m-%d'),'oven_id': oven_id, 'layer_id': layer_id, 'accumulation_hour': pre

2E0 finished


C:\Users\cherr\AppData\Local\Temp\ipykernel_16344\27521094.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_layer_processed['time_interval'] = df_layer_processed['time_interval'].dt.days
C:\Users\cherr\AppData\Local\Temp\ipykernel_16344\27521094.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_layer_processed['time_interval'] = df_layer_processed['time_interval'].dt.days
C:\Users\cherr\AppData\Local\Temp\ipykernel_16344\27521094.py:36: SettingWithCopyWarning: 
A value is trying to be set on a 

2G0 finished


In [14]:
def time_1(oven_id):
    input_path = os.path.join(current_directory, 'data/accmulation_hour_prediction', f'{oven_id}.csv')
    df = pd.read_csv(input_path)
    df = df.loc[:, ~df.columns.str.startswith('Unnamed:')]

    df['date'] = pd.to_datetime(df['date'])

    start_date = pd.to_datetime('2022-09-02')
    end_date = pd.to_datetime('2022-09-28')

    # filter
    df = df[(df['date'] >= start_date) & (df['date'] <= end_date)]
    df.to_csv(input_path)

In [15]:
for oven_id in oven_id_lst_p1:
    time_1(oven_id)

In [7]:
def time(oven_id):
    input_path = os.path.join(current_directory, 'data/accmulation_hour_prediction', f'{oven_id}.csv')
    df = pd.read_csv(input_path)
    df = df.loc[:, ~df.columns.str.startswith('Unnamed:')]

    df['date'] = pd.to_datetime(df['date'])

    start_date = pd.to_datetime('2023-02-07')
    end_date = pd.to_datetime('2023-02-28')

    # filter
    df = df[(df['date'] >= start_date) & (df['date'] <= end_date)]
    df.to_csv(input_path)

In [8]:
for oven_id in oven_id_lst_p2[:-1]:
    time(oven_id)

1. 依照使用時間添加上對應的temperature, power, water資料
2. 將損壞lamp_id添加進資料
3. 將各筆資料的lamp_id擴展

In [9]:
def p(oven_id):
    input_path = os.path.join(current_directory, 'data/accmulation_hour_prediction', f'{oven_id}.csv')
    df = pd.read_csv(input_path)
    
    cooler_path = os.path.join(current_directory, 'data/row_data', 'cooler.csv')
    cooler = pd.read_csv(cooler_path)

    # 將 cooler_id 中以 "temperature" 結尾的行選擇出來，形成 temperature_df
    temperature_df = cooler[cooler['cooler_id'].str.endswith('temperature')]

    # 將 cooler_id 中以 "water_volume" 結尾的行選擇出來，形成 water_volume_df
    water_volume_df = cooler[cooler['cooler_id'].str.endswith('water_volume')]

    # 重新設定索引
    temperature_df.reset_index(drop=True, inplace=True)
    water_volume_df.reset_index(drop=True, inplace=True)

    water_volume_df = water_volume_df.set_index('cooler_id')
    temperature_df = temperature_df.set_index('cooler_id')

    water_volumn = []
    temp_a = []
    temp_b = []

    for index, row in df.iterrows():
        id = int(row['layer_id'])
        volumn = (water_volume_df.loc[f'Slot{id}_water_volume'][oven_id] + water_volume_df.loc[f'Slot{id+1}_water_volume'][oven_id])/2
        water_volumn.append(volumn)
        a_temp = 0
        b_temp = 0
        if id == 19:
            a_temp = temperature_df.loc[f'S{id:02d}_A_temperature'][oven_id]
            b_temp = temperature_df.loc[f'S{id:02d}_B_temperature'][oven_id]

        else:
            a_temp = (temperature_df.loc[f'S{id:02d}_A_temperature'][oven_id] + temperature_df.loc[f'S{id+1:02d}_A_temperature'][oven_id])/2
            b_temp = (temperature_df.loc[f'S{id:02d}_B_temperature'][oven_id] + temperature_df.loc[f'S{id+1:02d}_B_temperature'][oven_id])/2
        temp_a.append(a_temp)
        temp_b.append(b_temp)
    dic = pd.DataFrame({
        'water_volum': water_volumn,
        'A_temperature': temp_a,
        'B_temperature': temp_b
    })

    df = pd.concat([df, dic], axis=1)

    power_path = os.path.join(current_directory, 'data/row_data', 'power.csv')
    power = pd.read_csv(power_path)

    bins = [0]
    for index, row in power.iterrows():
        bins.append(row["accumulation_hour"].split('-')[1])

    label_1 = list(power["power_setup(other_lamp)"])

    df["power"] = pd.cut(df["accumulation_hour"].values, bins=bins, labels=label_1)

    ## 輸出
    output_path = os.path.join(current_directory, 'data/acc_hour_prediction_withpc', f'{oven_id}.csv')
    df.to_csv(output_path)

In [16]:
for oven_id in oven_id_lst_p1:
    p(oven_id)

In [13]:
for oven_id in oven_id_lst_p2[:-1]:
    p(oven_id)